# Transfer learning

Using model trained on 6-class emotion dataset with Train: 16000, Val: 2000, Test: 2000. 

Perform transfer learning on SST-2 dataset as well as 4-class Emotion dataset.

Emotion dataset sentences are less clean in the sense, it contains multiple words in short-term and more informal terms included.

SST-2 are movie reviews which are much cleaner as compared to 4-class Emotion dataset

Train a model on 6-class emotion dataset

In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import numpy as np 
import matplotlib.pyplot as plt
import nltk
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, roc_curve, auc
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import os
from sklearn.metrics import balanced_accuracy_score
!pip install autocorrect
import pickle
from autocorrect import Speller
import re
import string
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
!pip install emoji
import emoji


     |████████████████████████████████| 622 kB 9.7 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=eb4b8bdc26efb5e7e1eaab415edaa16a8432812c189b8fff1fc50cafb03d8df3
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect
     |████████████████████████████████| 175 kB 7.9 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=c1a395032465e3a748681c983bcfcc2fca2c3cc957335b78d971a1cc62198827
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [2]:
nltk.download('punkt')
!pip install transformers==4.17 # this will also install tokenizers 
!pip install datasets

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 3.8 MB 7.4 MB/s 
     |████████████████████████████████| 895 kB 61.8 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
     |████████████████████████████████| 77 kB 4.4 MB/s 
     |████████████████████████████████| 6.6 MB 55.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 8.1 MB/s 
     |████████████████████████████████| 1.1 MB 48.3 MB/s 
     |████████████████████████████████| 212 kB 60.0 MB/s 
     |████████████████████████████████| 136 kB 18.9 MB/s 
     |████████████████████████████████| 127 kB 48.9 MB/s 
     |████████████████████████████████| 144 kB 58.2 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 271 kB 21.1 MB/

In [3]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [4]:
import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer, RobertaForSequenceClassification,RobertaModel, Trainer, TrainingArguments
from transformers import AdamW, get_linear_schedule_with_warmup
import datasets
print(transformers.__version__)

4.17.0


In [5]:
#Installing emot library
!pip install emot #Importing libraries
import re
from emot.emo_unicode import UNICODE_EMOJI

     |████████████████████████████████| 61 kB 24 kB/s 


Load 6-class emotion dataset

In [27]:
data_6_class = datasets.load_dataset('emotion')

Using custom data configuration default
Reusing dataset emotion (/root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
num_labels = len(np.unique(data_6_class['train']['label']))
print (f'Number of labels : {num_labels}')

Number of labels : 6


In [9]:
# % splits of labels in train/validation/test
from collections import OrderedDict, Counter
class OrderedCounter(Counter, dict):
    'Counter that remembers the order elements are first encountered'
    def __repr__(self):
        return '%s(%r)' % (self.__class__.__name__, dict(self))

    def __reduce__(self):
        return self.__class__, (dict(self),)
data_len_train = len(data_6_class['train']['label'])
# print (data_len_train)
data_len_val = len(data_6_class['validation']['label'])
data_len_test = len(data_6_class['test']['label'])
# print (OrderedCounter(data['train']['label'])[19])

for i in range(num_labels):
  print ('Train labels : {}, percentage: {:.2f}'.format(i,(OrderedCounter(data_6_class['train']['label'])[i]/data_len_train)*100))
  print ('Val labels : {}, percentage: {:.2f}'.format(i,(OrderedCounter(data_6_class['validation']['label'])[i]/data_len_val)*100))
  print ('Test labels : {}, percentage: {:.2f}'.format(i,(OrderedCounter(data_6_class['test']['label'])[i]/data_len_test)*100))


Train labels : 0, percentage: 29.16
Val labels : 0, percentage: 27.50
Test labels : 0, percentage: 29.05
Train labels : 1, percentage: 33.51
Val labels : 1, percentage: 35.20
Test labels : 1, percentage: 34.75
Train labels : 2, percentage: 8.15
Val labels : 2, percentage: 8.90
Test labels : 2, percentage: 7.95
Train labels : 3, percentage: 13.49
Val labels : 3, percentage: 13.75
Test labels : 3, percentage: 13.75
Train labels : 4, percentage: 12.11
Val labels : 4, percentage: 10.60
Test labels : 4, percentage: 11.20
Train labels : 5, percentage: 3.57
Val labels : 5, percentage: 4.05
Test labels : 5, percentage: 3.30


In [10]:
# Hyper-parameters
model_name = "roberta-base" # we can change this, and play with any model from HuggingFace! 
# BERT-base: "bert-base-cased" #Sensitive to lower and upper case
# BERT-large: "bert-large-cased"
# RoBERTa-base: "roberta-base"
# RoBERTa-large: "roberta-large"
# etc ... 
max_length = 50
batch_size = 32
d_in = 768 # bert-base / roberta-base
d_h = 512 # 512 
d_out = num_labels # 2 classes so 1 is enough
dropout = 0.1
freeze_pretrained = False

# Training and Optimizer HP
# Hyper-parameters
n_epochs = 10
lr = 3e-05
warmup = 0.06
max_gradient_norm = 10
weight_decay = 0.1

In [11]:
tokenizer = AutoTokenizer.from_pretrained( # robertatokenizer fast
    'roberta-base',
    cache_dir = "."
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [12]:
### config
config = AutoConfig.from_pretrained(
    model_name, 
    output_hidden_states = True,
    output_attention = False
) 

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [13]:
class EmotionsDataset_Original():

    def __init__(self, tokenizer, df,max_length):
        super(EmotionsDataset_Original, self).__init__()

        self.tokenizer = tokenizer
        self.max_seq_len = max_length
        self.input_ids, self.attention_mask,self.token_type_ids,self.label = self.get_input(df)
        

    def __len__(self):
        return len(self.label)
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens_seq) # convert to id
        input_ids += padding   # add padding
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len) # for identifying next sentence prediction
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids 

    def get_input(self,df):          
        tweet = df['text'].values
        label = df['label'].values
        tokens_seq = list(map(self.tokenizer.tokenize,tweet))
        result = list(map(self.trunate_and_pad, tokens_seq))
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]

        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(label).type(torch.long)
               )

    def __getitem__(self, item):
        return self.input_ids[item], self.attention_mask[item],self.token_type_ids[item],self.label[item]

In [14]:
class FinetuneModel(nn.Module):
    """Pretrained Model + classifier
    """
    def __init__(self, pretrained_model, d_in, d_h, d_out,dropout,freeze_pretrained=False,model_dir = None):
        super(FinetuneModel, self).__init__()

        # Instantiate BERT model
        
        self.pretrained_model = pretrained_model
        if model_dir != None:
            self.pretrained_model.load_state_dict(model_dir,strict = False)
            print ('Model Loaded!')
            

        # Instantiate a 2-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_in, d_h),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_h, d_out)
        )

        # Freeze the encoder model
        if freeze_pretrained:
            for param in self.pretrained_model.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        # Feed input to pretrained model
        pre_outputs = self.pretrained_model(input_ids = input_ids,attention_mask = attention_mask)
        hidden_state = pre_outputs[0]
        hidden_state = hidden_state[:,0,:]
        logits = self.classifier(hidden_state)
        return logits

Dataloader

In [15]:
data_train = datasets.Dataset.to_pandas(data_6_class["train"])
data_val = datasets.Dataset.to_pandas(data_6_class["validation"])
data_test = datasets.Dataset.to_pandas(data_6_class["test"])

train_dataset = EmotionsDataset_Original(tokenizer, data_train,max_length)
val_dataset = EmotionsDataset_Original(tokenizer, data_val,max_length)
test_dataset = EmotionsDataset_Original(tokenizer, data_test,max_length)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

Function to init model

In [34]:
def model_init(model_name,lr,weight_decay,batch_size,warmup,data,freeze_pretrained):
  pretrained_model_emotions = RobertaModel.from_pretrained(model_name,config=config)
  model = FinetuneModel(pretrained_model_emotions,d_in,d_h,d_out,dropout,freeze_pretrained = freeze_pretrained)
  model.to(device)
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
          {
                  'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                  'weight_decay':weight_decay
          },
          {
                  'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                  'weight_decay':0.0
          }
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr, betas = (0.9,0.98),eps = 1e-6)
  total_steps = int(n_epochs * (len(data["train"]["text"]) / batch_size))
  warmup_steps = int(warmup * total_steps)
  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps = warmup_steps, 
      num_training_steps = total_steps)
  
  return model,optimizer,scheduler


Train and Eval function

In [17]:
def set_seed(seed_value = 0):
    """Set seed for reproducibility.
    """
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(train_loader, model, optimizer, scheduler,loss_fct):
    model.train()
    train_losses = []
    correct_preds = 0
    pred_class = []
    actual_class = []
    tqdm_loader = tqdm(train_loader)
    for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in enumerate (tqdm_loader):
        model.zero_grad()
  

        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        segments = segments.to(device)
        labels = labels.to(device)

        logits = model(input_ids, attention_mask) # forward pass (batch_size, 1)
        loss = loss_fct(logits,labels)
        probs = nn.functional.softmax(logits,dim = -1)
        _,out_classes = torch.max(logits,dim = 1)
        ## for weighted accuracy
        pred_class.extend(out_classes.cpu().numpy())
        actual_class.extend(labels.cpu().numpy())

        correct = (out_classes == labels).sum()
        correct_preds += correct.item()
        train_losses.append(loss.item())

        loss.backward() # backward pass
        ## clip grad
        nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step() # weights update
        scheduler.step()
        optimizer.zero_grad()
        
    train_loss = np.mean(train_losses)
    epoch_accuracy = correct_preds /len(train_loader.dataset)
    # epoch_accuracy = balanced_accuracy_score(actual_class,pred_class)

    return train_loss,epoch_accuracy

In [18]:
from sklearn.metrics import roc_auc_score, f1_score
def eval_multiclass(loader, model,loss_fct):
    model.eval()
    val_losses = []
    val_probs = []
    acc_correct = 0
    iter = 0
    log_interval = 100
    sigmoid = nn.Sigmoid()
    log_interval = 50
    count = 0
    val_labels = []
    val_targets = []
    pred_class = []
    actual_class = []
  
    # threshold = torch.tensor([0.5]).to(device)
    # loader = tqdm(loader)
    for (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in loader:
        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        segments = segments.to(device)
        # print (attention_mask.shape,labels.shape)
        # print (segments.shape)
        
        with torch.no_grad():
            valid_logits = model(input_ids, attention_mask) # raw logits
            # print (f'logits = {valid_logits}')
            # print(f'labels = {labels}')
            valid_loss = loss_fct(valid_logits,labels) # loss
            valid_probs = nn.functional.softmax(valid_logits,dim = -1)
            _,out_classes = torch.max(valid_logits,dim = 1)
            # if count % log_interval == 0:
            #   print (f'labels = {labels}')
            #   print (f'predicted labels = {out_classes}')
            f1_target = torch.argmax(valid_probs,dim= 1)
            # print (f1_target)
            correct = (out_classes == labels).sum()
            acc_correct += correct.item()
            # weighted accuracy

            pred_class.extend(out_classes.cpu().numpy())
            actual_class.extend(labels.cpu().numpy())


            # print (valid_probs.shape)
            val_probs.extend(valid_probs[:,:].cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_losses.append(valid_loss.item())
            val_targets.extend(f1_target.cpu().numpy())
            count +=1
            # print (count)
    val_loss = np.mean(val_losses)
    val_probs = np.array(val_probs)
    val_labels = np.array(val_labels)
    val_targets = np.array(val_targets)
    valid_accuracy = acc_correct/(len(loader.dataset))
    # valid_accuracy = balanced_accuracy_score(actual_class,pred_class)

    
    val_auc = 100* roc_auc_score(val_labels, val_probs, multi_class='ovr')
    val_f1_score = f1_score(val_labels,val_targets,average = 'weighted')

    return val_loss, val_auc, val_targets, val_labels,valid_accuracy,val_f1_score

In [19]:
model_path = 'TF_model'
if not os.path.exists(model_path):
  os.makedirs(model_path)

In [20]:
### Evaluate on clean data
# set seed
loss_fct = nn.CrossEntropyLoss()
set_seed(seed_value = 0)
model,optimizer,scheduler = model_init(model_name,lr,weight_decay,batch_size,warmup,data_6_class,freeze_pretrained)
val_aucs = []
val_accs = []
best_score = 0.
patience_counter = 0
val_loss, val_auc, _, _, val_acc,val_f1_score= eval_multiclass(val_loader, model,loss_fct)
val_aucs.append(val_auc)
val_accs.append(val_acc)
print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}, f1_score : {:.2f}".format(val_loss, val_auc,val_acc*100,val_f1_score))
for epoch in range(1, n_epochs + 1):
    
    print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

    # Training round
    train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler,loss_fct)

    # Validation round
    val_loss, val_auc, _, _,val_acc,val_f1_score= eval_multiclass(val_loader, model,loss_fct)
    val_aucs.append(val_auc)

    print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
  
    print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, f1_score : {:.2f} (best ACC: {:.4f} at epoch {})"
    .format(val_loss, val_auc,val_acc*100,val_f1_score,np.max(np.array(val_acc)),np.argmax(np.array(val_acc*100))))

    print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
    if val_auc == np.max(np.array(val_aucs)):
        print("!! New best val AUC !!")
    if val_acc == np.max(np.array(val_acc)):
        print("!! New best val ACC !!")

## For Transfer learning
torch.save({'model_state_dict': model.pretrained_model.state_dict(), # save only base_model
            'optimizer':optimizer.state_dict(),
            'scheduler':scheduler.state_dict()},os.path.join(model_path,'TF_model.pt'))
print ('Model Saved to {}'.format(model_path))

## eval on test dataset
_, test_auc, test_probs, test_labels,test_acc,test_f1_score = eval_multiclass(test_loader, model,loss_fct)
print ("test AUC : {:.2f}".format(test_auc))
print ("test f1 score : {:.2f}".format(test_f1_score))
print ("test ACC : {:.2f}".format(test_acc*100))
    

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  Futu


Epoch 0 (before training) val loss: 1.8066, AUC: 52.02, ACC: 8.9000, f1_score : 0.01

New epoch, epoch 1 / 10


  0%|          | 0/500 [00:00<?, ?it/s]

Training: loss: 0.8681, accuracy 66.7000
Validation: loss: 0.2290, AUC: 99.32, ACC : 92.1000, f1_score : 0.92 (best ACC: 0.9210 at epoch 0)
lr : 2.872340425531915e-05
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 2 / 10


...

Training: loss: 0.0643, accuracy 96.6313
Validation: loss: 0.1510, AUC: 99.76, ACC : 93.7500, f1_score : 0.94 (best ACC: 0.9375 at epoch 0)
lr : 6.3829787234042555e-06
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/500 [00:00<?, ?it/s]

Training: loss: 0.0571, accuracy 97.1375
Validation: loss: 0.1689, AUC: 99.75, ACC : 93.7500, f1_score : 0.94 (best ACC: 0.9375 at epoch 0)
lr : 3.1914893617021277e-06
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/500 [00:00<?, ?it/s]

Training: loss: 0.0464, accuracy 97.7313
Validation: loss: 0.1728, AUC: 99.76, ACC : 93.5000, f1_score : 0.94 (best ACC: 0.9350 at epoch 0)
lr : 0.0
!! New best val ACC !!
Model Saved to TF_model
test AUC : 99.62
test f1 score : 0.93
test ACC : 92.95


In [28]:
n_check = len(data_6_class["test"]["text"])
total_examples = dict()
total_correct = dict()
for i in range(n_check):
    test_label = test_labels[i]
    prob = test_probs[i]
    total_examples[test_label] = total_examples.get(test_label,0) + 1
    if test_label == int(prob):
      total_correct[int(prob)] = total_correct.get(int(prob),0) + 1

for i in range(num_labels):
  print ('total class {} correct is : {:.2f}%'.format(i,(total_correct[i]/total_examples[i])*100))

total class 0 correct is : 97.76%
total class 1 correct is : 93.53%
total class 2 correct is : 88.68%
total class 3 correct is : 92.36%


# Transfer learning on 4-class dataset

Perform on cleaned tweets and compare results to original results

In [30]:
data_4_class = datasets.load_dataset('tweet_eval','emotion')

Reusing dataset tweet_eval (/root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
def model_init(model_name,lr,weight_decay,batch_size,warmup,data,freeze_pretrained):
  pretrained_model_emotions = RobertaModel.from_pretrained(model_name,config=config)
  model_dict = os.path.join(model_path,'TF_model.pt')
  checkpoint = torch.load(model_dict)
  model = FinetuneModel(pretrained_model_emotions,d_in,d_h,d_out,dropout,freeze_pretrained = freeze_pretrained,model_dir = checkpoint['model_state_dict']) 
  print ('Model loaded!')
  # freeze the pretrained_model and load dict
  model.to(device)
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
          {
                  'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                  'weight_decay':weight_decay
          },
          {
                  'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                  'weight_decay':0.0
          }
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr, betas = (0.9,0.98),eps = 1e-6)
  total_steps = int(n_epochs * (len(data["train"]["text"]) / batch_size))
  warmup_steps = int(warmup * total_steps)
  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps = warmup_steps, 
      num_training_steps = total_steps)
  
  return model,optimizer,scheduler


In [23]:
num_labels = len(np.unique(data_4_class['train']['label']))

In [24]:
tokenizer = AutoTokenizer.from_pretrained( # robertatokenizer fast
    'roberta-base',
    cache_dir = "."
)

Dataset class which cleans tweets

In [25]:
class CleanedEmotionsDataset():

    def __init__(self, tokenizer, df,max_length):
        super(CleanedEmotionsDataset, self).__init__()

        self.tokenizer = tokenizer
        self.max_seq_len = max_length
        self.input_ids, self.attention_mask,self.token_type_ids,self.label = self.get_input(df)
        

    def __len__(self):
        return len(self.label)
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens_seq) # convert to id
        input_ids += padding   # add padding
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len) # for identifying next sentence prediction
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids 

    # strip  @ with words and numbers
    def remove_pattern(self,input_txt):
        r = re.findall("@[\w]*", input_txt)
        for i in r:
            input_txt = re.sub(i, '', input_txt)
        input_txt = re.sub(r"\d+", "", input_txt)
        return input_txt  

    ## strip all @ and # with words following
    def strip_links(self,text):
        link_regex = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
        links = re.findall(link_regex, text)
        for link in links:
            text = text.replace(link[0], ', ')    
        return text
        
    def strip_all_entities(self,text):
        entity_prefixes = ['@','#']
        for separator in  string.punctuation:
            if separator not in entity_prefixes :
                text = text.replace(separator,' ')
        words = []
        for word in text.split():
            word = word.strip()
            if word:
                if word[0] not in entity_prefixes:
                    words.append(word)
        return ' '.join(words)

        # removing stopwords
    def removing_stopwords(self,text):
        text = repr(text)
        No_StopWords = [word for word in word_tokenize(text) if word.lower() not in stoplist ]
        # Convert list of tokens_without_stopwords to String type.
        words_string = ' '.join(No_StopWords)    
        return words_string

    ## lemmatization
    def lemmatization(self,text):
        # Converting words to their root forms
        lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
        return ' '.join(lemma)
    
    ## convert emoji to text
    def convert_emojis(self,text):
        for emot in UNICODE_EMOJI:
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
            text = text.replace('_', " ")
        return text

    def get_input(self,df):
        # df_tweet = self.clean_dataset(df['text'])
        ## clean off tags

        for i in range(len(df['text'])):
            df['text'][i] = df['text'][i].lower()
            df['text'][i] = self.remove_pattern(df['text'][i])
            df['text'][i] = self.convert_emojis(df['text'][i])
            # df['text'][i] = self.lemmatization(df['text'][i])
            # df['text'][i] = self.removing_stopwords(df['text'][i])
            # df['text'][i] = self.spell(self.lemmatization(df['text'][i]))
            
        tweet = df['text'].values
        # print (tweet)
        label = df['label'].values
        tokens_seq = list(map(self.tokenizer.tokenize,tweet))
        result = list(map(self.trunate_and_pad, tokens_seq))
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]

        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(label).type(torch.long)
               )

    def __getitem__(self, item):
        return self.input_ids[item], self.attention_mask[item],self.token_type_ids[item],self.label[item]

In [29]:
# Hyper-parameters
model_name = "roberta-base" # we can change this, and play with any model from HuggingFace! 
# BERT-base: "bert-base-cased" #Sensitive to lower and upper case
# BERT-large: "bert-large-cased"
# RoBERTa-base: "roberta-base"
# RoBERTa-large: "roberta-large"
# etc ... 
max_length = 50
batch_size = 32
d_in = 768 # bert-base / roberta-base
d_h = 512 # 512 
d_out = num_labels # 2 classes so 1 is enough
dropout = 0.1
freeze_pretrained = True

# Training and Optimizer HP
# Hyper-parameters
n_epochs = 10
lr = 3e-05
warmup = 0.06
max_gradient_norm = 10
weight_decay = 0.1

In [31]:
## lemmatization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
## speller
speller = Speller(lang = 'en')
# Convert to pandas df
data_train = datasets.Dataset.to_pandas(data_4_class["train"])
data_val = datasets.Dataset.to_pandas(data_4_class["validation"])
data_test = datasets.Dataset.to_pandas(data_4_class["test"])

train_dataset = CleanedEmotionsDataset(tokenizer, data_train,max_length)
val_dataset = CleanedEmotionsDataset(tokenizer, data_val,max_length)
test_dataset = CleanedEmotionsDataset(tokenizer, data_test,max_length)

# data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Evaluate Transfer learning

Try freezing pretrained model and not freezing

In [41]:
### Evaluate Transfer learning from 6-class to 4-class dataset
freeze_set = [True,False]
# set seed
loss_fct = nn.CrossEntropyLoss()
for freeze in freeze_set:
  freeze_pretrained = freeze # set freeze_pretrained
  set_seed(seed_value = 0)
  model,optimizer,scheduler = model_init(model_name,lr,weight_decay,batch_size,warmup,data_6_class,freeze_pretrained=freeze_pretrained)
  val_aucs = []
  val_accs = []
  best_score = 0.
  patience_counter = 0
  val_loss, val_auc, _, _, val_acc,val_f1_score= eval_multiclass(val_loader, model,loss_fct)
  val_aucs.append(val_auc)
  val_accs.append(val_acc)
  print ('Performing Transfer learning on 4-class Emotion dataset with freeze = {}'.format(str(freeze)))
  print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}, f1_score : {:.2f}".format(val_loss, val_auc,val_acc*100,val_f1_score))
  for epoch in range(1, n_epochs + 1):
      
      print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

      # Training round
      train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler,loss_fct)

      # Validation round
      val_loss, val_auc, _, _,val_acc,val_f1_score= eval_multiclass(val_loader, model,loss_fct)
      val_aucs.append(val_auc)

      print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
    
      print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, f1_score : {:.2f} (best ACC: {:.4f} at epoch {})"
      .format(val_loss, val_auc,val_acc*100,val_f1_score,np.max(np.array(val_acc)),np.argmax(np.array(val_acc*100))))

      print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
      if val_auc == np.max(np.array(val_aucs)):
          print("!! New best val AUC !!")
      if val_acc == np.max(np.array(val_acc)):
          print("!! New best val ACC !!")

  ## eval on test dataset
  _, test_auc, test_probs, test_labels,test_acc,test_f1_score = eval_multiclass(test_loader, model,loss_fct)
  print ("test AUC of {} freezed layers: {:.2f}".format(test_auc,freeze))
  print ("test f1 score of {} freezed layers: {:.2f}".format(test_f1_score,freeze))
  print ("test ACC of {} freezed layers: {:.2f}".format(test_acc*100,freeze))
    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model Loaded!
Model loaded!


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Performing Transfer learning on 4-class Emotion dataset with freeze = True

Epoch 0 (before training) val loss: 2.0918, AUC: 53.11, ACC: 32.6203, f1_score : 0.32

New epoch, epoch 1 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

...

Training: loss: 0.1048, accuracy 96.8683
Validation: loss: 0.8385, AUC: 92.94, ACC : 79.9465, f1_score : 0.80 (best ACC: 0.7995 at epoch 0)
lr : 2.6706382978723406e-05
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

Training: loss: 0.0870, accuracy 97.2367
Validation: loss: 1.0413, AUC: 92.52, ACC : 75.9358, f1_score : 0.77 (best ACC: 0.7594 at epoch 0)
lr : 2.605531914893617e-05
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

Training: loss: 0.0697, accuracy 98.0964
Validation: loss: 0.9606, AUC: 93.55, ACC : 79.6791, f1_score : 0.80 (best ACC: 0.7968 at epoch 0)
lr : 2.5404255319148937e-05
!! New best val ACC !!
test AUC of 93.76016802947204 freezed layers: 0.00
test f1 score of 0.7974889210810431 freezed layers: 0.00
test ACC of 79.52146375791696 freezed layers: 0.00


In [43]:
## check % of classes correct
n_check = len(data_4_class["test"]["text"])
total_examples = dict()
total_correct = dict()
for i in range(n_check):
    test_label = test_labels[i]
    prob = test_probs[i]
    total_examples[test_label] = total_examples.get(test_label,0) + 1
    if test_label == int(prob):
      total_correct[int(prob)] = total_correct.get(int(prob),0) + 1
# print (total_examples)
# print (total_correct)
for i in range(num_labels):
  print ('total class {} correct is : {:.2f}%'.format(i,(total_correct[i]/total_examples[i])*100))


total class 0 correct is : 78.85%
total class 1 correct is : 68.44%
total class 2 correct is : 4.88%
total class 3 correct is : 67.02%


Transfer learning on SST2 dataset

In [42]:
data_sst2 = datasets.load_dataset("SetFit/sst2")

Using custom data configuration SetFit--sst2-c66a8eb4897c1f8f


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/SetFit--sst2-c66a8eb4897c1f8f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [43]:
num_labels = len(np.unique(data_sst2['train']['label']))
num_labels

2

In [44]:
tokenizer = AutoTokenizer.from_pretrained( # robertatokenizer fast
    'roberta-base',
    cache_dir = "."
)

In [45]:
# Hyper-parameters
model_name = "roberta-base" # we can change this, and play with any model from HuggingFace! 
# BERT-base: "bert-base-cased" #Sensitive to lower and upper case
# BERT-large: "bert-large-cased"
# RoBERTa-base: "roberta-base"
# RoBERTa-large: "roberta-large"
# etc ... 
max_length = 50
batch_size = 32
d_in = 768 # bert-base / roberta-base
d_h = 512 # 512 
d_out = num_labels # 2 classes so 1 is enough
dropout = 0.1
freeze_pretrained = True

# Training and Optimizer HP
# Hyper-parameters
n_epochs = 10
lr = 3e-05
warmup = 0.06
max_gradient_norm = 10
weight_decay = 0.1

In [46]:
data_train = datasets.Dataset.to_pandas(data_sst2["train"])
data_val = datasets.Dataset.to_pandas(data_sst2["validation"])
data_test = datasets.Dataset.to_pandas(data_sst2["test"])

train_dataset = EmotionsDataset_Original(tokenizer, data_train,max_length)
val_dataset = EmotionsDataset_Original(tokenizer, data_val,max_length)
test_dataset = EmotionsDataset_Original(tokenizer, data_test,max_length)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [47]:
def eval_singleclass(loader, model,loss_fct):
  # compute auc for single class
    model.eval()
    val_losses = []
    val_probs = []
    val_targets = []
    acc_correct = 0
    iter = 0
    log_interval = 100
    sigmoid = nn.Sigmoid()

    val_labels = []
    # threshold = torch.tensor([0.5]).to(device)
    # loader = tqdm(loader)
    for (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in loader:
        
        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        segments = segments.to(device)
        # print (attention_mask.shape,labels.shape)

        with torch.no_grad():
            valid_logits = model(input_ids, attention_mask)
            valid_loss = loss_fct(valid_logits,labels)
            valid_probs = nn.functional.softmax(valid_logits,dim = -1)
            _,out_classes = valid_probs.max(dim =1)
            correct = (out_classes == labels).sum()
            f1_target = torch.argmax(valid_probs,dim= 1)
            acc_correct += correct.item()

            val_probs.extend(valid_probs[:,1].cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_losses.append(valid_loss.item())
            val_targets.extend(f1_target.cpu().numpy())

    val_loss = np.mean(val_losses)
    val_probs = np.array(val_probs)
    val_labels = np.array(val_labels)
    val_targets = np.array(val_targets)
    valid_accuracy = acc_correct/(len(loader.dataset))

    
    fpr, tpr, threshold = roc_curve(val_labels, val_probs)
    val_auc = 100 * auc(fpr, tpr)
    val_f1_score = f1_score(val_labels,val_targets)

    return val_loss, val_auc, val_probs, val_labels,valid_accuracy,val_f1_score

In [48]:
### Evaluate Transfer learning from 6-class to SST2 dataset
# set seed
loss_fct = nn.CrossEntropyLoss()
freeze_set = [True,False]
for freeze in freeze_set:
  freeze_pretrained = freeze
  set_seed(seed_value = 0)
  model,optimizer,scheduler = model_init(model_name,lr,weight_decay,batch_size,warmup,data_6_class,freeze_pretrained = freeze_pretrained)
  val_aucs = []
  val_accs = []
  best_score = 0.
  patience_counter = 0
  val_loss, val_auc, _, _, val_acc,val_f1_score= eval_singleclass(val_loader, model,loss_fct)
  val_aucs.append(val_auc)
  val_accs.append(val_acc)
  print ('Performing Transfer learning on SST2 dataset with freeze = {}'.format(str(freeze)))
  print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}, f1_score : {:.2f}".format(val_loss, val_auc,val_acc*100,val_f1_score))
  for epoch in range(1, n_epochs + 1):
      
      print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

      # Training round
      train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler,loss_fct)

      # Validation round
      val_loss, val_auc, _, _,val_acc,val_f1_score= eval_singleclass(val_loader, model,loss_fct)
      val_aucs.append(val_auc)

      print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
    
      print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, f1_score : {:.2f} (best ACC: {:.4f} at epoch {})"
      .format(val_loss, val_auc,val_acc*100,val_f1_score,np.max(np.array(val_acc)),np.argmax(np.array(val_acc*100))))

      print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
      if val_auc == np.max(np.array(val_aucs)):
          print("!! New best val AUC !!")
      if val_acc == np.max(np.array(val_acc)):
          print("!! New best val ACC !!")

  ## eval on test dataset
  _, test_auc, test_probs, test_labels,test_acc,test_f1_score = eval_singleclass(test_loader, model,loss_fct)
  print ("test AUC : {:.2f} with freeze = {}".format(test_auc,freeze))
  print ("test f1 score : {:.2f} with freeze = {}".format(test_f1_score,freeze))
  print ("test ACC : {:.2f} with freeze = {}".format(test_acc*100,freeze))
    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model Loaded!
Model loaded!


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Performing Transfer learning on SST2 dataset with freeze = True

Epoch 0 (before training) val loss: 0.7770, AUC: 75.93, ACC: 63.6468, f1_score : 0.72

New epoch, epoch 1 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

...

Training: loss: 0.0180, accuracy 99.4798
Validation: loss: 0.6745, AUC: 96.97, ACC : 90.2523, f1_score : 0.91 (best ACC: 0.9025 at epoch 0)
lr : 2.083404255319149e-05
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.0210, accuracy 99.4364
Validation: loss: 0.4751, AUC: 96.98, ACC : 92.0872, f1_score : 0.92 (best ACC: 0.9209 at epoch 0)
lr : 1.9448936170212766e-05
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.0275, accuracy 99.1474
Validation: loss: 0.4939, AUC: 97.35, ACC : 91.2844, f1_score : 0.91 (best ACC: 0.9128 at epoch 0)
lr : 1.8063829787234045e-05
!! New best val ACC !!
test AUC : 98.45 with freeze = False
test f1 score : 0.93 with freeze = False
test ACC : 93.52 with freeze = False
